Introduction:

Hoole, a prominent e-commerce platform specializing in urban-style piercings and jewelry, is dedicated to enhancing its comprehension of user interactions and behaviors within the last 30 days. This analysis of the provided dataset aims to uncover invaluable insights that can steer decisions related to marketing strategies, inventory management, and user experience enhancements.

The central objective of this analytical endeavor is to develop a profound comprehension of customer conduct on the UrbanStyle website. Through an exploration of user engagement, demographic trends, product preferences, and purchasing tendencies, we aspire to offer well-informed recommendations that can elevate user satisfaction, foster higher engagement, and optimize overarching business strategies.

Key Questions:

Key Questions:
1. How is the distribution of gender and age?
2. Which product categories are the most popular among users in terms purchases?
3. Are there certain price ranges that attract more user engagement?
4. What is the average price of both purchased and viewed products on the platform?
5. How does the average price vary across different genders, age groups, and categories?
6. Are there specific time periods when user activity peaks?
7. What types of events dominate the dataset?
8. How successful is the conversion of views to purchases for different product categories?
9. Is there a notable trend in how often customers return to make additional purchases?
10. How do the rates of adding products to the cart, converting carts to purchases, and abandoning carts provide insights into overall buying behavior?



--- 
 **Index:**
* [ Data Preprocessing and Exploration](#preprocessing)
#
* [1. User Demographics](#01)
     - [1.1. Distribution of Age](#1-1)
     - [1.2. Distribution of Gender](#1-2)
#
* [2. Product Preferences](#02)
     - [2.1. Popular Product Categories](#2-1)
     - [2.2. Purchased Products by Price Range](#2-2)
#
* [3. Pricing Analysis](#03)
     - [3.1. Average Prices](#3-1)
        - [3.1.1. Genders and Categories](#3-1-1)
        - [3.1.2. Age Groups and Categories](#3-1-2)
#
* [4. Customer Interaction Analysis](#04)
     - [4.1. Activity](#4-1)
     - [4.2. Events](#4-2)
#
* [5. Purchase Analysis](#05)
     - [5.1. Conversion Rate by Product Category](#5-1)
     - [5.2. Customer Loyalty & Repeat Transactions](#5-2)
     - [5.3. Cart Behavior Insights](#5-2-2)

---

<a id="preprocessing"></a>
# Data Preprocessing and Exploration

In [70]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from IPython.display import Image
from datetime import timedelta

df = pd.read_csv ('hoole.csv')

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3147 entries, 0 to 3146
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   event_time     3147 non-null   object 
 1   user_id        3147 non-null   int64  
 2   sex            3147 non-null   object 
 3   age            3147 non-null   int64  
 4   event_type     3147 non-null   object 
 5   product_id     3147 non-null   int64  
 6   category_code  3147 non-null   object 
 7   price          3147 non-null   float64
dtypes: float64(1), int64(3), object(4)
memory usage: 196.8+ KB


In [72]:
df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d')

df["week"] = df["event_time"].dt.day_name()
df['date'] = df['event_time'].dt.date
df['time'] = pd.to_datetime(df['event_time'], format='%H').dt.strftime('%H')

In [73]:
df.describe()

,user_id,age,product_id,price
count,3.147000e+03,3147.000000,3.147000e+03,3147.000000
mean,5.222331e+08,24.761042,5.236484e+06,26.234051
std,8.690602e+07,4.020778,1.736303e+06,17.390041
min,4.369600e+07,18.000000,4.569000e+03,1.590000
25%,4.822704e+08,21.000000,5.700825e+06,8.895000
50%,5.597553e+08,25.000000,5.819119e+06,26.690000
75%,5.835980e+08,28.000000,5.881776e+06,40.390000
max,5.953731e+08,32.000000,5.916125e+06,59.900000


In [74]:
df.head()

,event_time,user_id,sex,age,event_type,product_id,category_code,price,week,date,time
0,2023-05-01 02:33:14,534972827,female,24,view,5651975,Piercings.PiercingRings,17.19,Monday,2023-05-01,02
1,2023-05-01 02:38:30,532955590,female,23,view,5900052,Jewellery.ToeRings,34.99,Monday,2023-05-01,02
2,2023-05-01 02:41:20,532955590,female,23,purchase,5900052,Jewellery.ToeRings,34.99,Monday,2023-05-01,02
3,2023-05-01 05:22:23,513931240,female,19,cart,5304,Jewellery.Charms,31.79,Monday,2023-05-01,05
4,2023-05-01 05:22:24,513931240,female,19,cart,5304,Jewellery.Charms,31.79,Monday,2023-05-01,05


In [75]:
df.category_code.head(10)

0    Piercings.PiercingRings
1         Jewellery.ToeRings
2         Jewellery.ToeRings
3           Jewellery.Charms
4           Jewellery.Charms
5           Jewellery.Charms
6           Jewellery.Charms
7           Jewellery.Charms
8          Jewellery.Anklets
9           Jewellery.Charms
Name: category_code, dtype: object

In [76]:
# Split by the first period (n=1)
df[['main_category', 'sub_category']] = df['category_code'].str.split('.', n=1, expand=True)

In [77]:
df = df[[ 'event_time', 'date', 'time', 'week', 'user_id', 'sex', 'age', 'event_type', 'product_id', 'category_code', 'main_category', 'sub_category', 'price']]

<a id="01"></a>
# 1. User Demographics

<a id="1-1"></a>
### 1.1. Distribution of Age

In [78]:
# Create a DataFrame to count the distribution of 'age'
age_distribution = df['age'].value_counts().reset_index()
age_distribution.columns = ['age', 'count']
    

age_count = age_distribution['count'].sum()
age_distribution['percentage'] = (age_distribution['count'] / age_count) * 100

# Plot the distribution of 'age'
fig = px.bar(age_distribution, 
             x='age', 
             y='count', 
             title="Distribution of Age")
fig.show()

age_distribution

,age,count,percentage
0,25,447,14.204004
1,26,426,13.536702
2,19,376,11.947887
3,28,341,10.835717
4,31,338,10.740388
5,24,282,8.960915
6,20,253,8.039403
7,23,206,6.545917
8,18,148,4.702892
9,30,85,2.700985


<a id="1-2"></a>
### 1.2. Distribution of Sex

In [80]:
# Create a DataFrame to count the distribution of 'sex'
sex_distribution = df['sex'].value_counts().reset_index()
sex_distribution.columns = ['sex', 'count']

sex_distribution_count = sex_distribution['count'].sum()
sex_distribution['percentage'] = (sex_distribution['count'] / sex_distribution_count) * 100

# Plot the distribution of 'sex'
fig = px.bar(sex_distribution, 
             x='sex', 
             y='count', 
             title="Distribution of Sex")
fig.show()

sex_distribution

,sex,count,percentage
0,female,2694,85.605338
1,male,453,14.394662


The gender distribution analysis highlights a substantial majority of female users (85.61%) compared to male users (14.39%) on the platform. It's crucial to tailor marketing strategies and product offerings to address both genders for a balanced and engaging user experience.

<a id="02"></a>
# 2. Product Preferences

<a id="2-1"></a>
### 2.1. Category distribution

In [112]:
# Filter df for purchases only
purchase_df = df[df['event_type'] == 'purchase']

In [113]:
# Calculate main-category values
mainCategory_values = purchase_df.groupby('main_category')['product_id'].nunique().sort_values(ascending=False)
mainCategory_names = mainCategory_values.index

# Pie chart for main-categories
fig = px.pie(
    names=mainCategory_names, 
    values=mainCategory_values, 
    hole=0.3)
fig.update_layout(title={"text": "Main Category distribution", "x": 0.5})
fig.show()

print('Number of main-categories: ', len(mainCategory_names))
mainCategory_values

Number of main-categories:  2


main_category
Piercings    54
Jewellery    41
Name: product_id, dtype: int64

In [114]:
# Calculate sub-category values
subCategory_values = purchase_df['sub_category'].value_counts().sort_values(ascending=False)
subCategory_names = subCategory_values.index

# Create the pie chart
fig = px.pie(names=subCategory_names, 
             values=subCategory_values, 
             hole=0.3)
fig.update_layout(title={"text": "Sub Category distribution", "x": 0.5})
fig.show()

print('Number of sub-categories: ', len(subCategory_names))
subCategory_values

Number of sub-categories:  24


Charms                   44
Rings                    21
PiercingRings            17
ToeRings                 15
StretchingTools          14
Flatbeads                13
EarWeights&Hangers       13
Beads                     8
CurvedBarbells            8
Helix&Tragus              7
Tools&Accessories         5
Tunnels&Plugs             5
EarringsStuds&Shields     5
FingerClaws               4
Pendants                  4
Spirals                   4
FakePiercings             3
BallsPins&More            3
Necklaces                 3
Bracelets                 2
Barbells                  2
Labrets                   2
NoseJewellery&Septums     1
Anklets                   1
Name: sub_category, dtype: int64

In [117]:
# Assign variables
subCategory_values_top5 = subCategory_values[:5]
subCategory_names_top5 = subCategory_names[:5]
other_value = sum(subCategory_values[5:])

# Concatenate arrays and calculate the sum
subCategory_values_with_other = np.append(subCategory_values_top5, other_value)
subCategory_names_with_other = np.append(subCategory_names_top5, 'Other')

# Create the pie chart
fig = px.pie(names=subCategory_names_with_other, values=subCategory_values_with_other, hole=0.3)
fig.update_layout(title={"text": "Top 5 Sub-category distribution", "x": 0.5})
fig.show()
subCategory_values_top5


Charms             44
Rings              21
PiercingRings      17
ToeRings           15
StretchingTools    14
Name: sub_category, dtype: int64

<a id="2-2"></a>
### 2.2. Distribution of Products by Price Range

In [120]:
bins = [0, 5, 10, 20, 30, 40, 50, float('inf')]
labels = ['0-5 EUR', '5-10 EUR', '10-20 EUR', '20-30 EUR', '30-40 EUR', '40-50 EUR', 'Above 50 EUR']

price_range_counts = pd.cut(purchase_df['price'], bins=bins, labels=labels).value_counts().reset_index()
price_range_counts.columns = ['price', 'count']

# Create a donut chart using Plotly Express
fig = px.pie(
    price_range_counts, 
    values='count', 
    names='price', 
    hole=0.3,
    labels={'price': 'Price Range'},
    title='Distribution of Products by Price Range')

# Show the donut chart
fig.show()

price_range_counts


,price,count
0,30-40 EUR,66
1,20-30 EUR,35
2,0-5 EUR,28
3,Above 50 EUR,25
4,40-50 EUR,23
5,10-20 EUR,16
6,5-10 EUR,11


<a id="03"></a>
# 3. Pricing Analysis

<a id="3-1"></a>
### 3.1. Average Prices

In [122]:
# Calculate the average price of viewed products
average_viewed_price = df.loc[df['event_type'] == 'view', 'price'].mean()

# Calculate the average price of purchased products
average_purchased_price = df.loc[df['event_type'] == 'purchase', 'price'].mean()

# Print the results
print(f"Average Price of Viewed Products: ${average_viewed_price:.2f}")
print(f"Average Price of Purchased Products: ${average_purchased_price:.2f}")

Average Price of Viewed Products: $22.94
Average Price of Purchased Products: $28.98


<a id="3-1-1"></a>
### 3.1.1. Genders and Categories

In [123]:
# Calculate the average prices by gender and category
average_prices_gender_category = purchase_df.groupby(['sex', 'main_category'])['price'].mean().reset_index()

fig = px.bar(average_prices_gender_category, x='main_category', y='price', color='sex', barmode='group',
             title='Average Prices by Main Category and Sex',
             labels={'main_category': 'Main Category', 'price': 'Average Price'})

fig.update_yaxes(title_text='Average Price')
fig.show()

average_prices_gender_category


,sex,main_category,price
0,female,Jewellery,30.159815
1,female,Piercings,30.679000
2,male,Jewellery,22.707500
3,male,Piercings,16.779286


<a id="3-1-2"></a>
### 3.1.2. Age Groups and Categories

In [124]:
# Define constant variables
bins = [18, 20, 25, 30, 100]
labels = ['18-20', '20-25', '25-30', '30+']

# Filter the DataFrame and calculate average prices
purchase_data = df.loc[df['event_type'] == 'purchase'].copy()
purchase_data['age_group'] = pd.cut(purchase_data['age'], bins=bins, labels=labels, right=False)
average_prices_age_category = purchase_data.groupby(['age_group', 'main_category'])['price'].mean().reset_index()

# Create a grouped bar plot using Plotly Express
fig = px.bar(
    average_prices_age_category,
    x='main_category',
    y='price',
    color='age_group',
    title='Average Prices Comparison by Age Group and Main Category',
    labels={'age_group': 'Age Group', 'price': 'Average Price'},
    barmode='group'
)
fig.update_layout(xaxis={'categoryorder': 'total ascending'})
fig.show()

average_prices_age_category


,age_group,main_category,price
0,18-20,Jewellery,26.482273
1,18-20,Piercings,27.351667
2,20-25,Jewellery,26.977632
3,20-25,Piercings,26.022593
4,25-30,Jewellery,31.076522
5,25-30,Piercings,28.720526
6,30+,Jewellery,35.176429
7,30+,Piercings,37.175714


<a id="04"></a>
# 4. Customer Interaction Analysis

In [92]:
activity = df.groupby(['date','time'])['user_id'].agg(['count']).reset_index().sort_values(by=['date','time'])

activity['date'] = pd.to_datetime(activity['date'], format='%Y-%m-%d %H:%M:%S')

activity['time'] = pd.to_datetime(activity['time'], format='%H').dt.strftime('%H')

activity['day_of_week'] = activity['date'].dt.day_name()
activity['day'] = activity['date'].dt.day
activity

,date,time,count,day_of_week,day
0,2023-05-01,02,3,Monday,1
1,2023-05-01,05,5,Monday,1
2,2023-05-01,06,1,Monday,1
3,2023-05-01,07,4,Monday,1
4,2023-05-01,08,6,Monday,1
...,...,...,...,...,...
622,2023-05-31,17,2,Wednesday,31
623,2023-05-31,18,2,Wednesday,31
624,2023-05-31,19,3,Wednesday,31
625,2023-05-31,20,3,Wednesday,31


### 4.1. Activity

In [93]:
day_activity = activity.groupby('date')['count'].agg(['sum']).reset_index()
day_activity_count = day_activity['sum'].sum()
day_activity['percentage'] = (day_activity['sum'] / day_activity_count) * 100

fig = px.line(day_activity, 
              x='date', 
              y='sum', 
              title="Sum of Users Over the days")
fig.update_xaxes(tickmode='linear')
fig.show()


week_activity = activity.groupby('day_of_week')['count'].agg(['sum']).reset_index()
week_activity_count = week_activity['sum'].sum()
week_activity['percentage'] = (week_activity['sum'] / week_activity_count) * 100

fig = px.bar(week_activity, 
             x='day_of_week', 
             y='sum', 
             title="Sum of Users Over the days")
fig.update_xaxes(tickmode='linear')
fig.show()


hour_activity = activity.groupby('time')['count'].agg(['sum']).reset_index()
hour_activity_count = hour_activity['sum'].sum()
hour_activity['percentage'] = (hour_activity['sum'] / hour_activity_count) * 100

fig = px.bar(hour_activity, 
             x='time', 
             y='sum', 
             title="Sum of Users Over the days")
fig.update_xaxes(tickmode='linear')
fig.show()

### 4.2. Events

In [94]:
event_types = ['view', 'cart', 'purchase']

# Count events by day for each event type
event_by_day = df[df['event_type'].isin(event_types)].groupby(['date', 'event_type']).size().unstack()

# Plot bar chart
fig = px.bar(event_by_day, 
              x=event_by_day.index, 
              y=event_types,
             title="Distribution of Events by day",
             labels={"x": "day in a Day", "y": "Number of Events"},
             template="plotly_white")
fig.update_xaxes(tickmode='linear')

# Show the plot
fig.show()

event_by_day

event_type,cart,purchase,view
date,,,
2023-05-01,29.0,10.0,50.0
2023-05-02,40.0,7.0,90.0
2023-05-03,43.0,7.0,79.0
2023-05-04,27.0,5.0,62.0
2023-05-05,26.0,11.0,58.0
2023-05-06,44.0,13.0,57.0
2023-05-07,29.0,6.0,31.0
2023-05-08,34.0,9.0,44.0
2023-05-09,31.0,11.0,63.0


In [95]:
# Count events by week for each event type
event_by_week = df[df['event_type'].isin(event_types)].groupby(['week', 'event_type']).size().unstack()

# Plot bar chart
fig = px.bar(event_by_week, 
             x=event_by_week.index, 
             y=event_types,
             title="Distribution of Events by week",
             labels={"x": "week in a Day", "y": "Number of Events"},
             template="plotly_white")
fig.update_xaxes(tickmode='linear')

# Show the plot
fig.show()

event_by_week

event_type,cart,purchase,view
week,,,
Friday,104,37,199
Monday,126,37,210
Saturday,127,32,212
Sunday,94,24,152
Thursday,106,20,209
Tuesday,149,30,290
Wednesday,135,24,285


In [96]:
# Count events by hour for each event type
event_by_hour = df[df['event_type'].isin(event_types)].groupby(['time', 'event_type']).size().unstack()

# Plot bar chart
fig = px.bar(event_by_hour, 
              x=event_by_hour.index, 
              y=event_types,
             title="Distribution of Events by Hour",
             labels={"x": "Hour in a Day", "y": "Number of Events"},
             template="plotly_white")
fig.update_xaxes(tickmode='linear')


# Show the plot
fig.show()

event_by_hour

event_type,cart,purchase,view
time,,,
00,14.0,3.0,8.0
01,10.0,NaN,12.0
02,5.0,1.0,13.0
03,10.0,NaN,30.0
04,16.0,1.0,22.0
05,42.0,9.0,46.0
06,27.0,9.0,78.0
07,37.0,10.0,72.0
08,50.0,13.0,74.0


<a id="05"></a>
# 5. Purchase Analysis

Key Questions:
- What is the overall conversion rate from views to purchases?
- Do certain product categories or price ranges have higher conversion rates?
- How many existing customers made two or more purchases within 30 days.
- What are the Add-to-cart Rate (ATCR), Cart-to-checkout Rate (CTCR) and Cart Abandonment Rate (CAR).


<a id="5-1"></a>
### 5.1. Conversion Rate by Product Category


In [125]:
# Calculate conversion rates by product category
conversion_rates_by_category = df.groupby('main_category')['event_type'].value_counts(normalize=True).unstack()
conversion_rates_by_category['conversion_rate'] = conversion_rates_by_category['purchase'] / conversion_rates_by_category['view']

# Sort categories by conversion rate in descending order
conversion_rates_by_category = conversion_rates_by_category.sort_values(by='conversion_rate', ascending=False).reset_index()

# Plot the conversion rates
fig = px.bar(conversion_rates_by_category, x='main_category', y='conversion_rate', title="Conversion Rates by Product Category")
fig.show()

conversion_rates_by_category


event_type,main_category,cart,purchase,remove_from_cart,view,conversion_rate
0,Jewellery,0.331343,0.089552,0.188806,0.390299,0.229446
1,Piercings,0.219701,0.046486,0.161594,0.572219,0.081238


<a id="5-2"></a>
### 5.2. Customer Loyalty & Repeat Transactions


In [126]:
time_frame = timedelta(days=30)  # Change this as needed

# Filter out unique users who have made more than one purchase within the time frame
returning_customers = df[df['event_type'] == 'purchase'] \
    .groupby('user_id')['event_time'] \
    .apply(lambda x: (x.max() - x.min()) <= time_frame) \
    .reset_index(name='returning') \
    .query('returning == True')['user_id']

# Calculate the Returning Customer Rate
rcr = len(returning_customers) / len(df['user_id'].unique()) * 100

print(f"Returning Customer Rate: {rcr:.2f}%")

Returning Customer Rate: 12.43%


In [111]:
grouped_purchases = purchase_df.groupby('user_id')

repeat_transaction_intervals = []
for _, group in grouped_purchases:
    # Sort purchases by 'event_time' in place
    group.sort_values('event_time', inplace=True)
    
    # Calculate time differences between consecutive purchases and add to the list
    repeat_transaction_intervals.extend(group['event_time'].diff().dropna().dt.days.tolist())

# Calculate the average time between repeat transactions
average_time_between_transactions = sum(repeat_transaction_intervals) / len(repeat_transaction_intervals)

print(f"Average time between repeat transactions: {average_time_between_transactions:.2f} days")


Average time between repeat transactions: 1.49 days


<a id="5-3"></a>
### 5.3. Cart Behavior Insights


In [127]:
# Calculate Add-to-cart Rate (ATCR)
atcr_numerator = df[df['event_type'] == 'cart']['user_id'].nunique()
atcr_denominator = df[df['event_type'] == 'view']['user_id'].nunique()
atcr = (atcr_numerator / atcr_denominator * 100) if atcr_denominator != 0 else 0.0

# Calculate Cart-to-checkout Rate (CTCR)
ctcr_numerator = df[df['event_type'] == 'purchase']['user_id'].nunique()
ctcr_denominator = df[df['event_type'] == 'cart']['user_id'].nunique()
ctcr = (ctcr_numerator / ctcr_denominator * 100) if ctcr_denominator != 0 else 0.0

# Calculate Cart Abandonment Rate (CAR)
car_numerator = df[(df['event_type'] == 'cart') & (~df['user_id'].isin(df[df['event_type'] == 'purchase']['user_id']))]['user_id'].nunique()
car_denominator = df[df['event_type'] == 'cart']['user_id'].nunique()
car = (car_numerator / car_denominator * 100) if car_denominator != 0 else 0.0

# Calculate Cart Removal Rate (CRR)
crr_numerator = df[df['event_type'] == 'remove_from_cart']['user_id'].nunique()
crr_denominator = df[df['event_type'] == 'cart']['user_id'].nunique()
crr = (crr_numerator / crr_denominator * 100) if crr_denominator != 0 else 0.0

print(f"Add-to-cart Rate (ATCR): {atcr:.2f}%")
print(f"Cart-to-checkout Rate (CTCR): {ctcr:.2f}%")
print(f"Cart Abandonment Rate (CAR): {car:.2f}%")
print(f"Cart Removal Rate (CRR): {crr:.2f}%")

Add-to-cart Rate (ATCR): 53.55%
Cart-to-checkout Rate (CTCR): 31.53%
Cart Abandonment Rate (CAR): 72.20%
Cart Removal Rate (CRR): 49.81%


In [102]:
# Create a DataFrame for the rates with rounded values
rates_data = pd.DataFrame({
    'Rate Type': ['Add-to-cart Rate (ATCR)', 'Cart-to-checkout Rate (CTCR)', 'Cart Abandonment Rate (CAR)'],
    'Rate': [round(rate, 2) for rate in [atcr, ctcr, car]]
})

# Create a bar chart using Plotly Express
fig = px.bar(rates_data, x='Rate Type', y='Rate', title='Conversion Rates',
             labels={'Rate Type': 'Rate Type', 'Rate': 'Rate (%)'})

fig.show()

rates_data


,Rate Type,Rate
0,Add-to-cart Rate (ATCR),53.55
1,Cart-to-checkout Rate (CTCR),31.53
2,Cart Abandonment Rate (CAR),72.20
